<a href="https://colab.research.google.com/github/SudipBhakta/Vehicle_usage_forecast/blob/main/Vehicle_Alocation_Product_Wise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np


# ==========================================
# LOAD DATA
# ==========================================

CSV_PATH = "/content/Vehilce_Prediction_Demo_Data.csv"
df = pd.read_csv(CSV_PATH)

df["Year"] = df["Year"].astype(int)
df["Month"] = df["Month"].astype(int)

df["Total_Demand"] = df["Product_Quantity"] * df["Vehicle_Usage"]


# ==========================================
# VEHICLE TYPE YoY GROWTH
# ==========================================

def calculate_vehicle_growth(product_name):

    product_df = df[df["Product"] == product_name]

    yearly = (
        product_df.groupby(["Year", "Vehicle_Type"])["Total_Demand"]
        .sum()
        .reset_index()
    )

    yearly = yearly.sort_values(["Vehicle_Type", "Year"])

    yearly["YoY_Growth"] = yearly.groupby("Vehicle_Type")[
        "Total_Demand"
    ].pct_change()

    latest_growth = (
        yearly.groupby("Vehicle_Type")
        .tail(1)
        .set_index("Vehicle_Type")["YoY_Growth"]
        .fillna(0)
    )

    return latest_growth


# ==========================================
# ROUTE YoY GROWTH
# ==========================================

def calculate_route_growth(product_name):

    product_df = df[df["Product"] == product_name].copy()

    product_df["Route_Key"] = (
        product_df["Plant"] + "_" +
        product_df["Destination"]
    )

    yearly = (
        product_df.groupby(["Year", "Route_Key"])["Total_Demand"]
        .sum()
        .reset_index()
    )

    yearly = yearly.sort_values(["Route_Key", "Year"])

    yearly["YoY_Growth"] = yearly.groupby("Route_Key")[
        "Total_Demand"
    ].pct_change()

    latest_growth = (
        yearly.groupby("Route_Key")
        .tail(1)
        .set_index("Route_Key")["YoY_Growth"]
        .fillna(0)
    )

    return latest_growth


# ==========================================
# FUTURE ALLOCATION FUNCTION
# ==========================================

def allocate_future(year, month, demand_p1, demand_p2):

    final_rows = []

    for product_name, total_demand in [("P1", demand_p1), ("P2", demand_p2)]:

        product_df = df[df["Product"] == product_name].copy()

        # Seasonal filter
        seasonal_df = product_df[product_df["Month"] == month].copy()

        seasonal_df["Full_Route"] = (
            seasonal_df["Plant"] + "_" +
            seasonal_df["Destination"] + "_" +
            seasonal_df["Vehicle_Type"]
        )

        seasonal_df["Route_Key"] = (
            seasonal_df["Plant"] + "_" +
            seasonal_df["Destination"]
        )

        # Base seasonal share
        route_total = (
            seasonal_df.groupby("Full_Route")["Total_Demand"]
            .sum()
        )

        base_share = route_total / route_total.sum()

        # Get growth factors
        vehicle_growth = calculate_vehicle_growth(product_name)
        route_growth = calculate_route_growth(product_name)

        adjusted_share = {}

        for route, share in base_share.items():

            plant, destination, vehicle_type = route.split("_")
            route_key = plant + "_" + destination

            v_growth = vehicle_growth.get(vehicle_type, 0)
            r_growth = route_growth.get(route_key, 0)

            # Combined growth effect
            adjusted_share[route] = share * (1 + v_growth) * (1 + r_growth)

        # Normalize again
        total_adj = sum(adjusted_share.values())

        for route in adjusted_share:
            adjusted_share[route] /= total_adj

        # Allocate demand
        for route, share in adjusted_share.items():

            plant, destination, vehicle_type = route.split("_")

            route_key = plant + "_" + destination

            route_demand = total_demand * share

            capacity = product_df[
                (product_df["Plant"] == plant) &
                (product_df["Destination"] == destination) &
                (product_df["Vehicle_Type"] == vehicle_type)
            ]["Product_Quantity"].iloc[0]

            vehicles_needed = np.ceil(route_demand / capacity)

            final_rows.append({
                "Year": year,
                "Month": month,
                "Product": product_name,
                "Plant": plant,
                "Destination": destination,
                "Vehicle_Type": vehicle_type,
                "Route_YoY_Growth": round(route_growth.get(route_key, 0), 3),
                "Vehicle_YoY_Growth": round(vehicle_growth.get(vehicle_type, 0), 3),
                "Adjusted_Share": round(share, 4),
                "Route_Demand": round(route_demand, 2),
                "Vehicles_Allocated": int(vehicles_needed),
                "Allocated_Quantity": int(vehicles_needed * capacity)
            })

    return pd.DataFrame(final_rows)


# ==========================================
# USER INPUT
# ==========================================

future_year = int(input("\nEnter Future Year: "))
future_month = int(input("Enter Future Month (1-12): "))
future_demand_p1 = float(input("Enter TOTAL Demand for P1: "))
future_demand_p2 = float(input("Enter TOTAL Demand for P2: "))

result = allocate_future(
    future_year,
    future_month,
    future_demand_p1,
    future_demand_p2
)

print("\nVehicle Allocation Result (Seasonal + Route Growth + Vehicle Growth):")
print(result)

print("\nSummary Check:")
print(result.groupby("Product")[["Route_Demand", "Allocated_Quantity"]].sum())


Enter Future Year: 2023
Enter Future Month (1-12): 5
Enter TOTAL Demand for P1: 9856
Enter TOTAL Demand for P2: 12451

Vehicle Allocation Result (Seasonal + Route Growth + Vehicle Growth):
    Year  Month Product Plant Destination Vehicle_Type  Route_YoY_Growth  \
0   2023      5      P1    B1          D1         17FT             0.041   
1   2023      5      P1    B1          D1         19FT             0.041   
2   2023      5      P1    B1          D1         22FT             0.041   
3   2023      5      P1    B1          D1         32FT             0.041   
4   2023      5      P1    B1          D2         17FT            -0.018   
5   2023      5      P1    B1          D2         19FT            -0.018   
6   2023      5      P1    B1          D2         22FT            -0.018   
7   2023      5      P1    B1          D2         32FT            -0.018   
8   2023      5      P1    B2          D3         17FT             0.081   
9   2023      5      P1    B2          D3         

In [18]:
import pandas as pd
import numpy as np


# ==========================================
# LOAD DATA
# ==========================================

CSV_PATH = "/content/Vehilce_Prediction_Demo_Data.csv"
df = pd.read_csv(CSV_PATH)

df["Year"] = df["Year"].astype(int)
df["Month"] = df["Month"].astype(int)

df["Total_Demand"] = df["Product_Quantity"] * df["Vehicle_Usage"]


# ==========================================
# VEHICLE TYPE YoY GROWTH
# ==========================================

def calculate_vehicle_growth(product_name):

    product_df = df[df["Product"] == product_name]

    yearly = (
        product_df.groupby(["Year", "Vehicle_Type"])["Total_Demand"]
        .sum()
        .reset_index()
    )

    yearly = yearly.sort_values(["Vehicle_Type", "Year"])

    yearly["YoY_Growth"] = yearly.groupby("Vehicle_Type")[
        "Total_Demand"
    ].pct_change()

    latest_growth = (
        yearly.groupby("Vehicle_Type")
        .tail(1)
        .set_index("Vehicle_Type")["YoY_Growth"]
        .fillna(0)
    )

    return latest_growth


# ==========================================
# ROUTE YoY GROWTH
# ==========================================

def calculate_route_growth(product_name):

    product_df = df[df["Product"] == product_name].copy()

    product_df["Route_Key"] = (
        product_df["Plant"] + "_" +
        product_df["Destination"]
    )

    yearly = (
        product_df.groupby(["Year", "Route_Key"])["Total_Demand"]
        .sum()
        .reset_index()
    )

    yearly = yearly.sort_values(["Route_Key", "Year"])

    yearly["YoY_Growth"] = yearly.groupby("Route_Key")[
        "Total_Demand"
    ].pct_change()

    latest_growth = (
        yearly.groupby("Route_Key")
        .tail(1)
        .set_index("Route_Key")["YoY_Growth"]
        .fillna(0)
    )

    return latest_growth


# ==========================================
# FUTURE ALLOCATION FUNCTION
# ==========================================

def allocate_future(year, month, demand_p1, demand_p2):

    final_rows = []

    for product_name, total_demand in [("P1", demand_p1), ("P2", demand_p2)]:

        product_df = df[df["Product"] == product_name].copy()

        # Seasonal filter
        seasonal_df = product_df[product_df["Month"] == month].copy()

        seasonal_df["Full_Route"] = (
            seasonal_df["Plant"] + "_" +
            seasonal_df["Destination"] + "_" +
            seasonal_df["Vehicle_Type"]
        )

        seasonal_df["Route_Key"] = (
            seasonal_df["Plant"] + "_" +
            seasonal_df["Destination"]
        )

        # Base seasonal share
        route_total = (
            seasonal_df.groupby("Full_Route")["Total_Demand"]
            .sum()
        )

        base_share = route_total / route_total.sum()

        # Get growth factors
        vehicle_growth = calculate_vehicle_growth(product_name)
        route_growth = calculate_route_growth(product_name)

        adjusted_share = {}

        for route, share in base_share.items():

            plant, destination, vehicle_type = route.split("_")
            route_key = plant + "_" + destination

            v_growth = vehicle_growth.get(vehicle_type, 0)
            r_growth = route_growth.get(route_key, 0)

            # Combined growth effect
            adjusted_share[route] = share * (1 + v_growth) * (1 + r_growth)

        # Normalize again
        total_adj = sum(adjusted_share.values())

        for route in adjusted_share:
            adjusted_share[route] /= total_adj

        # Allocate demand
        for route, share in adjusted_share.items():

            plant, destination, vehicle_type = route.split("_")

            route_key = plant + "_" + destination

            route_demand = total_demand * share

            capacity = product_df[
                (product_df["Plant"] == plant) &
                (product_df["Destination"] == destination) &
                (product_df["Vehicle_Type"] == vehicle_type)
            ]["Product_Quantity"].iloc[0]

            vehicles_needed = np.ceil(route_demand / capacity)

            final_rows.append({
                "Year": year,
                "Month": month,
                "Product": product_name,
                "Plant": plant,
                "Destination": destination,
                "Vehicle_Type": vehicle_type,
                # "Route_YoY_Growth": round(route_growth.get(route_key, 0), 3),
                # "Vehicle_YoY_Growth": round(vehicle_growth.get(vehicle_type, 0), 3),
                # "Adjusted_Share": round(share, 4),
                # "Route_Demand": round(route_demand, 2),
                "Vehicles_Allocated": int(vehicles_needed),
                "Allocated_Quantity": int(vehicles_needed * capacity)
            })

    return pd.DataFrame(final_rows)


# ==========================================
# USER INPUT
# ==========================================

future_year = int(input("\nEnter Future Year: "))
future_month = int(input("Enter Future Month (1-12): "))
future_demand_p1 = float(input("Enter TOTAL Demand for P1: "))
future_demand_p2 = float(input("Enter TOTAL Demand for P2: "))

result = allocate_future(
    future_year,
    future_month,
    future_demand_p1,
    future_demand_p2
)

print("\nVehicle Allocation Result (Seasonal + Route Growth + Vehicle Growth):")
print(result)

print("\nSummary Check:")
print(result.groupby("Product")[[ "Allocated_Quantity"]].sum())


Enter Future Year: 2026
Enter Future Month (1-12): 2
Enter TOTAL Demand for P1: 10342
Enter TOTAL Demand for P2: 14532

Vehicle Allocation Result (Seasonal + Route Growth + Vehicle Growth):
    Year  Month Product Plant Destination Vehicle_Type  Vehicles_Allocated  \
0   2026      2      P1    B1          D1         17FT                  77   
1   2026      2      P1    B1          D1         19FT                 105   
2   2026      2      P1    B1          D1         22FT                  68   
3   2026      2      P1    B1          D1         32FT                  84   
4   2026      2      P1    B1          D2         17FT                  65   
5   2026      2      P1    B1          D2         19FT                 103   
6   2026      2      P1    B1          D2         22FT                  56   
7   2026      2      P1    B1          D2         32FT                  79   
8   2026      2      P1    B2          D3         17FT                 100   
9   2026      2      P1    B2